In [ ]:
# default_exp data.features

# Featurizing Time Series

> Functions used to transform time series into a dataframe that can be used to create tabular dataloaders.

In [ ]:
#export
try: 
    from tsfresh import extract_features
    from tsfresh.feature_extraction.settings import ComprehensiveFCParameters, MinimalFCParameters, EfficientFCParameters
except ImportError:
    print("You need to install tsfresh to be able to import tsai.data.features")

In [ ]:
#export
from tsai.imports import *
from tsai.utils import *
from tsai.data.external import *
from tsai.data.core import *
from tsai.data.tabular import *

In this case we are using tsfresh that is one of the most widely known libraries used to create features from time series. You can get more details about this library here: https://tsfresh.readthedocs.io/en/latest/

In [ ]:
#export
def get_ts_features(X:Union[np.ndarray, torch.Tensor], y:Union[None, np.ndarray, torch.Tensor]=None, features:Union[str, dict]='min', 
                    n_jobs:Optional[int]=None, **kwargs):
    """
    Args:
        X: np.array or torch.Tesnor of shape [samples, dimensions, timesteps].
        y: Not required for unlabeled data. Otherwise, you need to pass it.
        features: 'min', 'efficient', 'all', or a dictionary. Be aware that 'efficient' and 'all' may required substantial memory and time.
    """    

    df = to_tsfresh_df(X)
    n_jobs = ifnone(n_jobs, defaults.cpus)
    if 'default_fc_parameters' in kwargs.keys(): default_fc_parameters = default_fc_parameters 
    elif features == 'min': default_fc_parameters = MinimalFCParameters()
    elif features == 'efficient': default_fc_parameters = EfficientFCParameters()
    elif features == 'all': default_fc_parameters = ComprehensiveFCParameters()
    else: default_fc_parameters = None
    df = extract_features(df, column_id="id", n_jobs=n_jobs, default_fc_parameters=default_fc_parameters, **kwargs)
    if y is not None:
        if y.ndim == 1: y = y.reshape(-1,1)
        for i in range(y.shape[-1]):
            df['target' if y.shape[-1] == 1 else f'target_{i}'] = y[:, i]
    return df

In [ ]:
#extras
dsid = 'NATOPS'
X, y, splits = get_UCR_data(dsid, return_split=False)
X.shape

(360, 24, 51)

There are 3 levels of fatures you can extract: 'min', 'efficient' and 'all'. I'd encourage you to start with min as feature creation may take a long time. 

In addition to this, you can pass a dictionary to build the desired features (see tsfresh documentation in the link above).

In [ ]:
#extras
ts_features_df = get_ts_features(X, y)
ts_features_df.shape

Feature Extraction: 100%|█████████████████████████████████| 40/40 [00:06<00:00,  6.22it/s]


(360, 217)

The 'min' set creates a dataframe with 8 features per channel + 1 per target (total 193) for each time series sample (360).

In [ ]:
#extras
cont_names = ts_features_df.columns[:-1]
y_names = 'target'
dls = get_tabular_dls(ts_features_df, splits=splits, cont_names=cont_names, y_names=y_names)
dls.show_batch()

,0__sum_values,0__median,0__mean,0__length,0__standard_deviation,0__variance,0__root_mean_square,0__maximum,0__minimum,1__sum_values,1__median,1__mean,1__length,1__standard_deviation,1__variance,1__root_mean_square,1__maximum,1__minimum,2__sum_values,2__median,2__mean,2__length,2__standard_deviation,2__variance,2__root_mean_square,2__maximum,2__minimum,3__sum_values,3__median,3__mean,3__length,3__standard_deviation,3__variance,3__root_mean_square,3__maximum,3__minimum,4__sum_values,4__median,4__mean,4__length,4__standard_deviation,4__variance,4__root_mean_square,4__maximum,4__minimum,5__sum_values,5__median,5__mean,5__length,5__standard_deviation,5__variance,5__root_mean_square,5__maximum,5__minimum,6__sum_values,6__median,6__mean,6__length,6__standard_deviation,6__variance,6__root_mean_square,6__maximum,6__minimum,7__sum_values,7__median,7__mean,7__length,7__standard_deviation,7__variance,7__root_mean_square,7__maximum,7__minimum,8__sum_values,8__median,8__mean,8__length,8__standard_deviation,8__variance,8__root_mean_square,8__maximum,8__minimum,9__sum_values,9__median,9__mean,9__length,9__standard_deviation,9__variance,9__root_mean_square,9__maximum,9__minimum,10__sum_values,10__median,10__mean,10__length,10__standard_deviation,10__variance,10__root_mean_square,10__maximum,10__minimum,11__sum_values,11__median,11__mean,11__length,11__standard_deviation,11__variance,11__root_mean_square,11__maximum,11__minimum,12__sum_values,12__median,12__mean,12__length,12__standard_deviation,12__variance,12__root_mean_square,12__maximum,12__minimum,13__sum_values,13__median,13__mean,13__length,13__standard_deviation,13__variance,13__root_mean_square,13__maximum,13__minimum,14__sum_values,14__median,14__mean,14__length,14__standard_deviation,14__variance,14__root_mean_square,14__maximum,14__minimum,15__sum_values,15__median,15__mean,15__length,15__standard_deviation,15__variance,15__root_mean_square,15__maximum,15__minimum,16__sum_values,16__median,16__mean,16__length,16__standard_deviation,16__variance,16__root_mean_square,16__maximum,16__minimum,17__sum_values,17__median,17__mean,17__length,17__standard_deviation,17__variance,17__root_mean_square,17__maximum,17__minimum,18__sum_values,18__median,18__mean,18__length,18__standard_deviation,18__variance,18__root_mean_square,18__maximum,18__minimum,19__sum_values,19__median,19__mean,19__length,19__standard_deviation,19__variance,19__root_mean_square,19__maximum,19__minimum,20__sum_values,20__median,20__mean,20__length,20__standard_deviation,20__variance,20__root_mean_square,20__maximum,20__minimum,21__sum_values,21__median,21__mean,21__length,21__standard_deviation,21__variance,21__root_mean_square,21__maximum,21__minimum,22__sum_values,22__median,22__mean,22__length,22__standard_deviation,22__variance,22__root_mean_square,22__maximum,22__minimum,23__sum_values,23__median,23__mean,23__length,23__standard_deviation,23__variance,23__root_mean_square,23__maximum,23__minimum,target
0,-26.125748,-0.617197,-0.512270,51.0,0.943701,0.890571,1.073774,1.078415,-1.911673,-34.008759,0.149119,-0.666838,51.0,1.255401,1.576032,1.421515,0.835408,-2.468259,-54.771980,-1.170985,-1.073960,51.0,0.654869,0.428854,1.257873,0.191203,-2.414801,45.089352,0.785930,0.884105,51.0,0.809274,0.654924,1.198568,2.188559,-0.348977,-43.958305,-0.416730,-0.861928,51.0,1.243240,1.545645,1.512800,0.828457,-2.767821,-6.500301,-0.273865,-0.127457,51.0,0.692578,0.479664,0.704208,1.395830,-1.101830,-32.830414,-0.733551,-0.643734,51.0,0.371831,0.138258,0.743405,0.026682,-1.213494,-13.595337,0.037761,-0.266575,51.0,0.551894,0.304587,0.612903,0.337016,-1.050993,-38.439110,-0.703090,-0.753708,51.0,0.165370,0.027347,0.771637,-0.536640,-1.041002,43.024174,0.858496,0.843611,51.0,0.234376,0.054932,0.875564,1.316007,0.467817,-33.416515,-0.767974,-0.655226,51.0,0.372972,0.139108,0.753942,0.107275,-1.236332,14.994287,0.265541,0.294006,51.0,0.329174,0.108356,0.441356,0.714693,-0.565684,-28.675285,-0.679008,-0.562261,51.0,0.830140,0.689133,1.0026

In [ ]:
#extras
x_cat, x_cont, yb = first(dls.train)
x_cont[:10]

tensor([[-0.3744,  0.5992, -0.3744,  ...,  1.4033,  2.1748, -0.4065],
        [-0.3864, -0.1467, -0.3864,  ...,  0.8158,  0.3099, -1.5182],
        [-0.0497,  0.0742, -0.0497,  ..., -0.4977, -0.3961,  0.4800],
        ...,
        [ 0.2529,  0.5441,  0.2529,  ..., -0.5928, -0.7302,  0.6398],
        [ 0.8455,  0.3421,  0.8455,  ..., -0.3040, -0.0638, -0.1428],
        [ 0.5438,  0.8348,  0.5438,  ..., -0.4219,  1.5014,  1.7773]])

In [ ]:
#hide
from tsai.imports import *
from tsai.export import *
nb_name = get_nb_name()
# nb_name = "020_data.features.ipynb"
create_scripts(nb_name);